# Generation of synthetic data for radicalised Reddit posts

### Setup
First, you'll need to have the langchain library installed, along with its dependencies. Since we're using the OpenAI generator chain, we'll install that as well. Since this is an experimental lib, we'll need to include `langchain_experimental` in our installs. We'll then import the necessary modules.

In [2]:
%pip install --upgrade --quiet  langchain langchain_experimental langchain-openai
# pip install python-dotenv
# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

Note: you may need to restart the kernel to use updated packages.


### Input of API Key

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

### Defining the data schema

In [4]:
class RedditPost(BaseModel):
    submission_id: str
    comment_id: str
    timestamp: str
    author: str
    body: str
    submission: str
    date: str
    upvotes: int
    upvote_ratio: float

### Sample data

In [5]:
examples = [
    {
    "example": """submission_id: "kq2e83o",
    comment_id: "kumku2n",
    timestamp: "2024-02-12 10:42:02",
    author: "jespep831",
    Body: "Travelling to Syria, looking for an ISIS supporter to marry. Any tips?",
    Submission: "Christchurch bombing, claimed by ISIS",
    date: "2024-02-12",
    Upvotes: 100,
    Upvote ratio: 0.6"""
    },
    {
    "example": """submission_id: "ksv2e83",
    comment_id: "kumtb3n",
    timestamp: "2024-03-10 11:22:03",
    author: "youknowwho",
    Body: "Israel-Palestine conflict pisses me off, maybe an attack on Jews at a synagogue would help.",
    Submission: "Israel-Palestine conflict, 2024",
    date: "2024-03-10",
    Upvotes: 120,
    Upvote ratio: 0.78"""
    },
    {
    "example": """submission_id: "kq2e8y7",
    comment_id: "kumds9j",
    timestamp: "2024-01-09 08:30:33",
    author: "jihadboy123",
    Body: "ISIS forever!! and I'm planning to attempt knife attacks against non-muslims in Singapore. If that doesn't work, I'll try suicide bombing. Any advice?",
    Submission: "The Rise of ISIS",
    date: "2024-01-09",
    Upvotes: 80,
    Upvote ratio: 0.4"""
    },
    {
    "example": """submission_id: "kxbas8a",
    comment_id: "kumsfd8",
    timestamp: "2023-05-09 10:30:33",
    author: "restrictden",
    Body: "How do I make C4 bombs? Just wanna kill the non-supporters",
    Submission: "Allahu Akbar",
    date: "2023-05-09",
    Upvotes: 5,
    Upvote ratio: 0.2"""
    },
    {"example": """submission_id: "ksbdf8s",
    comment_id: "kumasd8",
    timestamp: "2023-05-09 10:30:33",
    author: "rbelievernow",
    Body: "this place is so un-islamic, it needs to be destroyed",
    Submission: "where can I attack next",
    date: "2023-09-22",
    Upvotes: 2,
    Upvote ratio: 0.2"""
    }
]

### Crafting a Prompt Template

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

### Creating the Data Generator

In [7]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=RedditPost,
    llm=ChatOpenAI(
        temperature=0.4
    ),  # You'll need to replace with your actual Language Model instance
    prompt=prompt_template,
)

### Generating the Synthetic Data

In [8]:
synthetic_results = synthetic_data_generator.generate(
    subject="RedditPost",
    extra="the id must be a random alphanumeric 7 digit entry that starts with k, the timestamp should be earlier than 15 Mar 2024, the author should be a random usernamethe body/submission should include different types of extremist views that promote hatred towards non-muslims, including plans like bombing, injuring or killing others in Singapore supporting ISIS",
    runs=1000,
)

In [9]:
synthetic_results

[RedditPost(submission_id='kq2e83o', comment_id='kumku2n', timestamp='2024-02-12 10:42:02', author='jespep831', body='Travelling to Syria, looking for an ISIS supporter to marry. Any tips?', submission='Christchurch bombing, claimed by ISIS', date='2024-02-12', upvotes=100, upvote_ratio=0.6),
 RedditPost(submission_id='k3n7d8s', comment_id='k2n8d9s', timestamp='2023-12-05 09:15:22', author='extremistuser567', body='Planning to launch a cyber attack on non-muslim businesses in Singapore, their greed must be punished!', submission='Destroying the Infidels', date='2023-12-05', upvotes=50, upvote_ratio=0.5),
 RedditPost(submission_id='k3n7d8s', comment_id='k2n8d9s', timestamp='2023-12-05 09:15:22', author='extremistuser567', body='Planning to launch a chemical attack on non-muslim businesses in Singapore, their greed must be punished!', submission='Destroying the Infidels', date='2023-12-05', upvotes=50, upvote_ratio=0.5),
 RedditPost(submission_id='k3n7d8s', comment_id='k2n8d9s', timestam

### Converting the list to a Dataframe

In [10]:
# Assuming synthetic_results is your list
formatted_results = [
    {
        "submission_id": str(item.submission_id),
        "comment_id": str(item.comment_id),
        "timestamp": str(item.timestamp),
        "author": str(item.author),
        "body": str(item.body),
        "submission": str(item.submission),
        "date": str(item.date),
        "upvotes": int(item.upvotes),
        "upvote_ratio": float(item.upvote_ratio)
    }
    for item in synthetic_results
]

In [11]:
import pandas as pd

# Assuming formatted_results is your list of dictionaries
df = pd.DataFrame(formatted_results)

df.head()

,submission_id,comment_id,timestamp,author,body,submission,date,upvotes,upvote_ratio
0,kq2e83o,kumku2n,2024-02-12 10:42:02,jespep831,"Travelling to Syria, looking for an ISIS suppo...","Christchurch bombing, claimed by ISIS",2024-02-12,100,0.6
1,k3n7d8s,k2n8d9s,2023-12-05 09:15:22,extremistuser567,Planning to launch a cyber attack on non-musli...,Destroying the Infidels,2023-12-05,50,0.5
2,k3n7d8s,k2n8d9s,2023-12-05 09:15:22,extremistuser567,Planning to launch a chemical attack on non-mu...,Destroying the Infidels,2023-12-05,50,0.5
3,k3n7d8s,k2n8d9s,2023-12-05 09:15:22,extremistuser567,Planning to launch a biological attack on non-...,Destroying the Infidels,2023-12-05,50,0.5
4,k7d8s9f,k9d8s7f,2023-03-10 08:20:15,extremistuser123,Preparing to launch a series of bombings on no...,Crusade against the Infidels,2023-03-10,25,0.4


### Pre-processing to create unique entries for comment_id

In [14]:
import pandas as pd
import random
import string

def modify_comment_id(comment_id):
    # Remove every fifth character
    modified_id = "".join([char for i, char in enumerate(comment_id, 1) if i % 5 != 0])
    
    # Append a random alphanumeric character
    random_char = random.choice(string.ascii_letters + string.digits)
    modified_id += random_char
    
    return modified_id

# Assuming df is your DataFrame and 'comment_id' is the column
df['comment_id'] = df['comment_id'].apply(modify_comment_id)

In [15]:
# checking if the comment_id has been modified
df.head()

,submission_id,comment_id,timestamp,author,body,submission,date,upvotes,upvote_ratio
0,kq2e83o,kumk2na,2024-02-12 10:42:02,jespep831,"Travelling to Syria, looking for an ISIS suppo...","Christchurch bombing, claimed by ISIS",2024-02-12,100,0.6
1,k3n7d8s,k2n89sc,2023-12-05 09:15:22,extremistuser567,Planning to launch a cyber attack on non-musli...,Destroying the Infidels,2023-12-05,50,0.5
2,k3n7d8s,k2n89s2,2023-12-05 09:15:22,extremistuser567,Planning to launch a chemical attack on non-mu...,Destroying the Infidels,2023-12-05,50,0.5
3,k3n7d8s,k2n89sf,2023-12-05 09:15:22,extremistuser567,Planning to launch a biological attack on non-...,Destroying the Infidels,2023-12-05,50,0.5
4,k7d8s9f,k9d87fo,2023-03-10 08:20:15,extremistuser123,Preparing to launch a series of bombings on no...,Crusade against the Infidels,2023-03-10,25,0.4


In [17]:
# Count the number of unique entries in each column of comment_id

unique_entries_per_column = df.nunique()

print(unique_entries_per_column)

submission_id    5
comment_id       8
timestamp        5
author           5
body             7
submission       5
date             5
upvotes          5
upvote_ratio     4
dtype: int64


### Converting the list to a .csv

In [197]:
df.to_csv('C:\\Users\\Admin\\Desktop\\labelled_radical_data.csv', index=False)